In [1]:
import spotipy

In [2]:
secrets_file = open("..\spoti_secret.txt","r")

In [3]:
string = secrets_file.read()

In [4]:
string.split('\n')

['cid e9384bf289334c5ba89bba547e19ebc8',
 'csecret afa55186cd484c1bbc101c7401c5005f']

In [5]:
secrets_dict={}
for line in string.split('\n'):
    if len(line) > 0:
        secrets_dict[line.split(' ')[0]]=line.split(' ')[1]

In [6]:
secrets_dict

{'cid': 'e9384bf289334c5ba89bba547e19ebc8',
 'csecret': 'afa55186cd484c1bbc101c7401c5005f'}

#### Authentication with secrets

In [7]:
from spotipy.oauth2 import SpotifyClientCredentials

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=secrets_dict['cid'],
                                                           client_secret=secrets_dict['csecret']))

#### Searching songs with 'queries' with sp.search

In [8]:
from random import randint
from time import sleep

def get_playlist_tracks(playlist_id):
    results = sp.user_playlist_tracks("spotify",playlist_id)
    tracks = results['items']
    while results['next']!=None:
        results = sp.next(results)
        tracks = tracks + results['items']
        sleep(randint(1,3000)/1000)
    return tracks

#first we need to get a list of individual tracks from the playlist
def get_name_artist_from_playlist_item(playlist_item):
    return get_name_artists_from_track(playlist_item['track'])

def flatten(input_list):
    return [item for sublist in input_list for item in sublist]

def get_name_artists_from_playlist(input_playlist):
    return flatten(list(map(get_name_artist_from_playlist_item,input_playlist["items"])))

def get_name_artists_from_playlist2(input_playlist):
    return flatten(list(map(get_name_artist_from_playlist_item,input_playlist)))

def get_name_artists_from_track(track):
    return [(track["name"],artist["name"]) for artist in track["artists"]]

In [9]:
all_tracks = get_playlist_tracks("5S8SJdl1BDc0ugpkEvFsIL")
len(all_tracks)

10000

In [10]:
all_tracks[0]['track']['album']

{'album_type': 'single',
 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/06HL4z0CvFAxyc27GXpf02'},
   'href': 'https://api.spotify.com/v1/artists/06HL4z0CvFAxyc27GXpf02',
   'id': '06HL4z0CvFAxyc27GXpf02',
   'name': 'Taylor Swift',
   'type': 'artist',
   'uri': 'spotify:artist:06HL4z0CvFAxyc27GXpf02'}],
 'available_markets': [],
 'external_urls': {'spotify': 'https://open.spotify.com/album/0HG8fMDhvN2tH5uPHFsyZP'},
 'href': 'https://api.spotify.com/v1/albums/0HG8fMDhvN2tH5uPHFsyZP',
 'id': '0HG8fMDhvN2tH5uPHFsyZP',
 'images': [{'height': 640,
   'url': 'https://i.scdn.co/image/ab67616d0000b2734322e9bd7d57d061d0e19e1f',
   'width': 640},
  {'height': 300,
   'url': 'https://i.scdn.co/image/ab67616d00001e024322e9bd7d57d061d0e19e1f',
   'width': 300},
  {'height': 64,
   'url': 'https://i.scdn.co/image/ab67616d000048514322e9bd7d57d061d0e19e1f',
   'width': 64}],
 'name': '...Ready For It?',
 'release_date': '2017-09-03',
 'release_date_precision': 'day',
 'to

In [11]:
VideojuegosBienChingones = sp.user_playlist_tracks("spotify", "5t3KgspcXFc70ZABCc6N28")

In [12]:
VideojuegosBienChingones

{'href': 'https://api.spotify.com/v1/playlists/5t3KgspcXFc70ZABCc6N28/tracks?offset=0&limit=100&additional_types=track',
 'items': [{'added_at': '2019-03-13T10:40:49Z',
   'added_by': {'external_urls': {'spotify': 'https://open.spotify.com/user/jtafernandez'},
    'href': 'https://api.spotify.com/v1/users/jtafernandez',
    'id': 'jtafernandez',
    'type': 'user',
    'uri': 'spotify:user:jtafernandez'},
   'is_local': False,
   'primary_color': None,
   'track': {'album': {'album_type': 'compilation',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/0LyfQWJT6nXafLPZqxe9Of'},
       'href': 'https://api.spotify.com/v1/artists/0LyfQWJT6nXafLPZqxe9Of',
       'id': '0LyfQWJT6nXafLPZqxe9Of',
       'name': 'Various Artists',
       'type': 'artist',
       'uri': 'spotify:artist:0LyfQWJT6nXafLPZqxe9Of'}],
     'available_markets': [],
     'external_urls': {'spotify': 'https://open.spotify.com/album/3zSvk9thQMwQDrKMD0kJJv'},
     'href': 'https://api.spotif

In [13]:
VideojuegosBienChingones.keys()

dict_keys(['href', 'items', 'limit', 'next', 'offset', 'previous', 'total'])

In [14]:
VideojuegosBienChingones["items"][0].keys()

dict_keys(['added_at', 'added_by', 'is_local', 'primary_color', 'track', 'video_thumbnail'])

In [15]:
VideojuegosBienChingones["items"][0]["track"]

{'album': {'album_type': 'compilation',
  'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/0LyfQWJT6nXafLPZqxe9Of'},
    'href': 'https://api.spotify.com/v1/artists/0LyfQWJT6nXafLPZqxe9Of',
    'id': '0LyfQWJT6nXafLPZqxe9Of',
    'name': 'Various Artists',
    'type': 'artist',
    'uri': 'spotify:artist:0LyfQWJT6nXafLPZqxe9Of'}],
  'available_markets': [],
  'external_urls': {'spotify': 'https://open.spotify.com/album/3zSvk9thQMwQDrKMD0kJJv'},
  'href': 'https://api.spotify.com/v1/albums/3zSvk9thQMwQDrKMD0kJJv',
  'id': '3zSvk9thQMwQDrKMD0kJJv',
  'images': [{'height': 640,
    'url': 'https://i.scdn.co/image/ab67616d0000b273a448ea1a88acc7cf5bb26ed1',
    'width': 640},
   {'height': 300,
    'url': 'https://i.scdn.co/image/ab67616d00001e02a448ea1a88acc7cf5bb26ed1',
    'width': 300},
   {'height': 64,
    'url': 'https://i.scdn.co/image/ab67616d00004851a448ea1a88acc7cf5bb26ed1',
    'width': 64}],
  'name': 'Dark Souls 3',
  'release_date': '2016-01-01',
  '

In [16]:
get_name_artists_from_track(VideojuegosBienChingones["items"][0]["track"])

[('Dark Souls 3', 'Yuka Kitamura')]

In [17]:
get_name_artists_from_track(all_tracks[0]['track'])

[('...Ready For It?', 'Taylor Swift')]

In [18]:
lista = get_name_artists_from_playlist(VideojuegosBienChingones)

In [19]:
lista2 = get_name_artists_from_playlist2(all_tracks)

In [20]:
# VideojuegosBienChingones.keys()

In [21]:
# VideojuegosBienChingones['total']

In [22]:
# VideojuegosBienChingones['limit']

In [23]:
# VideojuegosBienChingones['next']

In [24]:
# sp.next(VideojuegosBienChingones)

In [25]:
VideojuegosBienChingones['offset']

0

In [26]:
VideojuegosBienChingones['previous'] # En este caso está vacío

In [27]:
import pandas as pd
df = pd.DataFrame(lista, columns = ['name','artist'])
df2 = pd.DataFrame(lista2, columns = ['name','artist'])

In [28]:
df.tail()

,name,artist
115,Hydrogen,M|O|O|N
116,"Ballad of the Goddess (From ""Skyward Sword"")",Taylor Davis
117,Turbo Killer,Carpenter Brut
118,House of the Rising Sun,The Animals
119,Soviet Connection (Theme from Gta Iv),Michael Hunter


In [29]:
df2

,name,artist
0,...Ready For It?,Taylor Swift
1,Life Changes,Thomas Rhett
2,24K Magic,Bruno Mars
3,Galway Girl,Ed Sheeran
4,Photograph,Ed Sheeran
...,...,...
12173,Fool's Overture,Supertramp
12174,Heart of the Sunrise - 2003 Remaster,Yes
12175,Venus And Mars / Rock Show / Jet - Live / Rema...,Paul McCartney
12176,Venus And Mars / Rock Show / Jet - Live / Rema...,Wings


In [30]:
for i in range(9999):
    print(df2['name'][i], df2['artist'][i])

...Ready For It? Taylor Swift
Life Changes Thomas Rhett
24K Magic Bruno Mars
Galway Girl Ed Sheeran
Photograph Ed Sheeran
Havana (feat. Young Thug) Camila Cabello
Havana (feat. Young Thug) Young Thug
When I Was Your Man Bruno Mars
Gonna Fly Now Bill Conti
Don't Wanna Know (feat. Kendrick Lamar) Maroon 5
Don't Wanna Know (feat. Kendrick Lamar) Kendrick Lamar
Thinking out Loud Ed Sheeran
Everybody Talks Neon Trees
We Are Never Ever Getting Back Together Taylor Swift
Bad Blood Taylor Swift
Hall of Fame (feat. will.i.am) The Script
Hall of Fame (feat. will.i.am) will.i.am
Payphone Maroon 5
Payphone Wiz Khalifa
Sugar Maroon 5
Stuck Like Glue Sugarland
That's What I Like Bruno Mars
Steal My Girl One Direction
I'm the One DJ Khaled
I'm the One Justin Bieber
I'm the One Quavo
I'm the One Chance the Rapper
I'm the One Lil Wayne
Wildest Dreams Taylor Swift
Young Dumb & Broke Khalid
Unwritten Natasha Bedingfield
Ignition - Remix R. Kelly
Free Fallin' Tom Petty
Brown Eyed Girl Van Morrison
It Girl

Cold Water Justin Bieber
Cold Water MØ
Me And My Broken Heart Rixton
New Man Ed Sheeran
Cake By The Ocean DNCE
Treasure Bruno Mars
Honey, I'm Good. Andy Grammer
Safe And Sound Capital Cities
Payphone Maroon 5
Payphone Wiz Khalifa
Want to Want Me Jason Derulo
I'm the One (feat. Justin Bieber, Quavo, Chance the Rapper & Lil Wayne) DJ Khaled
I'm the One (feat. Justin Bieber, Quavo, Chance the Rapper & Lil Wayne) Justin Bieber
I'm the One (feat. Justin Bieber, Quavo, Chance the Rapper & Lil Wayne) Quavo
I'm the One (feat. Justin Bieber, Quavo, Chance the Rapper & Lil Wayne) Chance the Rapper
I'm the One (feat. Justin Bieber, Quavo, Chance the Rapper & Lil Wayne) Lil Wayne
Billionaire (feat. Bruno Mars) Travie McCoy
Billionaire (feat. Bruno Mars) Bruno Mars
These Girls Why Don't We
Honey, I'm Good. Andy Grammer
Rain Is A Good Thing Luke Bryan
Bottoms Up Brantley Gilbert
Jumpman Drake
Jumpman Future
Don't Stop Believin' Journey
Twist And Shout - Remastered 2009 The Beatles
Jessie's Girl Rick

Kiss Prince
Maneater Daryl Hall & John Oates
What I Like About You The Romantics
Whip It DEVO
Never Gonna Give You Up Rick Astley
Danger Zone - From "Top Gun" Original Soundtrack Kenny Loggins
Keep on Loving You REO Speedwagon
Wake Me Up Before You Go-Go - Remastered Wham!
Your Love The Outfield
Rock You Like a Hurricane - 2011 Scorpions
I Ran A Flock Of Seagulls
Take Me Home Tonight Eddie Money
You Spin Me Round (Like a Record) Dead Or Alive
9 to 5 Dolly Parton
Break My Stride Matthew Wilder
Somebody's Watching Me - Single Version Rockwell
Nothing's Gonna Stop Us Now Starship
How Will I Know Whitney Houston
You Can Call Me Al Paul Simon
Material Girl Madonna
Talking In Your Sleep The Romantics
Working for the Weekend Loverboy
(I've Had) The Time of My Life Bill Medley
(I've Had) The Time of My Life Jennifer Warnes
Freeway of Love - Single Mix Aretha Franklin
Rock the Casbah - Remastered The Clash
Last Christmas Wham!
You Got It Roy Orbison
Take My Breath Away - Love Theme from "Top Gu

At The Cross (Love Ran Red) Chris Tomlin
Indescribable - Arriving Album Version Chris Tomlin
Gracefully Broken Matt Redman
Gracefully Broken Tasha Cobbs Leonard
Roxanne - Remastered 2003 The Police
Vineyard Jackopierce
Space Jam Quad City DJ's
Start Me Up - Remastered 2009 The Rolling Stones
God's Plan Drake
Gonna Fly Now - Theme From "Rocky" / Remastered 2006 Bill Conti
I Know What You Did Last Summer Shawn Mendes
I Know What You Did Last Summer Camila Cabello
Sugar, We're Goin Down Fall Out Boy
My Songs Know What You Did In The Dark (Light Em Up) Fall Out Boy
The Last Of The Real Ones Fall Out Boy
Big Girls Don't Cry (Personal) Fergie
Yes - From "Dirty Dancing" Soundtrack Merry Clayton
More Girls Like You Kip Moore
Hurricane Luke Combs
I Could Use a Love Song Maren Morris
Hometown Girl Josh Turner
Every Storm (Runs Out Of Rain) Gary Allan
Stay A Little Longer Brothers Osborne
Talladega Eric Church
Highway Don't Care Tim McGraw
Highway Don't Care Taylor Swift
Highway Don't Care Keith 

My Happy Ending Avril Lavigne
Starships Nicki Minaj
Rock Your Body Justin Timberlake
Stronger Kanye West
GDFR (feat. Sage the Gemini & Lookas) Flo Rida
GDFR (feat. Sage the Gemini & Lookas) Sage The Gemini
GDFR (feat. Sage the Gemini & Lookas) Lookas
Blurred Lines Robin Thicke
Blurred Lines T.I.
Blurred Lines Pharrell Williams
Just Can't Get Enough - 2006 Digital Remaster Depeche Mode
Lean On (feat. MØ & DJ Snake) Major Lazer
Lean On (feat. MØ & DJ Snake) MØ
Lean On (feat. MØ & DJ Snake) DJ Snake
Use Somebody Kings of Leon
Flaws Bastille
Mirror Lil Wayne
Mirror Bruno Mars
Where Them Girls At (feat. Nicki Minaj & Flo Rida) David Guetta
All Time Low - Recorded at Spotify Studios NYC Andy Grammer
Don't Say Goodnight Hot Chelle Rae
Good Life OneRepublic
Avalanche WALK THE MOON
Could This Be Love The Wanted
Cecilia And The Satellite Andrew McMahon in the Wilderness
Fine By Me Andy Grammer
Sunset Lover Petit Biscuit
Despacito - Remix Luis Fonsi
Despacito - Remix Daddy Yankee
Despacito - Remi

...

Oh Love Green Day
Kill the DJ Green Day
Stray Heart Green Day
Brutal Love Green Day
Nuclear Family Green Day
99 Revolutions Green Day
Brother Matt Corby
Bang Bang (My Baby Shot Me Down) Nancy Sinatra
Another Brick In The Wall, Pt. 2 - 2011 Remastered Version Pink Floyd
Somewhere Only We Know Keane
Future Starts Slow The Kills
CASTLE OF GLASS Linkin Park
IN MY REMAINS Linkin Park
Burning in the Skies Linkin Park
When They Come for Me Linkin Park
Waiting for the End Linkin Park
Blackout Linkin Park
Iridescent Linkin Park
The Messenger Linkin Park
A LIGHT THAT NEVER COMES - Rick Rubin Reboot Linkin Park
A LIGHT THAT NEVER COMES - Rick Rubin Reboot Rick Rubin
Beat The Devil's Tattoo Black Rebel Motorcycle Club
Come with Me Now KONGOS
Wheel in the Sky Journey
Tears Of An Angel RyanDan
The Swan Song Within Temptation
Angels The xx
Breathe Me Sia
Arrival of the Birds The Cinematic Orchestra
Arrival of the Birds London Metropolitan Orchestra
Into Dust Mazzy Star
Major Minus Coldplay
Broken Cro

We Are Never Apart Nick Mulvey
This Year Ine Hoem
Oh No! MARINA
Life on Mars? - 2015 Remaster David Bowie
Save Me Röyksopp
Save Me Susanne Sundfør
I Had This Thing Röyksopp
I Had This Thing Jamie Irrepressible
Kamikaze WALK THE MOON
Fire Anna of the North
Big Picture London Grammar
Backbeat Dagny
Wearing Nothing Dagny
Awake Sonny Alven
Awake Dragonette
Sunrise (feat. Jason Walker) Kygo
Sunrise (feat. Jason Walker) Jason Walker
On The Loose Niall Horan
Pumped Up Klingande
Not Alone Otto Knows
Listen Nico & Vinz
Hard To Get Trevis
Homemade Dynamite (Feat. Khalid, Post Malone & SZA) - REMIX Lorde
Homemade Dynamite (Feat. Khalid, Post Malone & SZA) - REMIX Khalid
Homemade Dynamite (Feat. Khalid, Post Malone & SZA) - REMIX Post Malone
Homemade Dynamite (Feat. Khalid, Post Malone & SZA) - REMIX SZA
Tessellate alt-J
Naken Kajander
Nye Joggesko Gabrielle
More Than a Feeling Boston
Don't You (Forget About Me) Simple Minds
Drive My Car - Remastered 2009 The Beatles
Loveshack (Live) The B-52's
Th

You Keaton Henson
When You Break Bear's Den
Peanut Butter Jelly Galantis
Buckle Up Join The Riot
Hello Adele
Sweet Dreams BØRNS
Alone - Lost Frequencies Remix Alan Walker
Alone - Lost Frequencies Remix Lost Frequencies
Up We Go Lights
Fire N Gold Bea Miller
Can't Feel My Face The Weeknd
Achy Breaky Heart Billy Ray Cyrus
Greased Lightnin' - From “Grease” Soundtrack John Travolta
Greased Lightnin' - From “Grease” Soundtrack Jeff Conaway
Summer Nights - From “Grease” Soundtrack John Travolta
Summer Nights - From “Grease” Soundtrack Olivia Newton-John
The Hardest Part Coldplay
Beautiful Sunday Daniel Boone
Show Me Love Sam Feldt
Show Me Love Kimberly Anne
Here for You (feat. Ella Henderson) Kygo
Here for You (feat. Ella Henderson) Ella Henderson
Rolling With Me (I Got Love) (feat. Maverick Sabre) SoulCircuit
Rolling With Me (I Got Love) (feat. Maverick Sabre) Maverick Sabre
Spiders - Finnebassen Remix Gundelach
Spiders - Finnebassen Remix Finnebassen
West Coast - The GRADES Icon Mix Lana D

Harry in Winter Patrick Doyle
The Hanging Tree James Newton Howard
The Hanging Tree Jennifer Lawrence
Day One Hans Zimmer
Go the Distance (Instrumental) [From "Hercules"] London Music Works
The Prince Of Persia Harry Gregson-Williams
Battle at Aslan's How Harry Gregson-Williams
The Call Regina Spektor
The Phantom of the Opera - Single Album Version Andrew Lloyd Webber
Fly Mel Ashton
Intouchables (Una Mattina) Jonas Kvarnström
Cantina Band John Williams
Cantina Band London Symphony Orchestra
Night At The Museum Alan Silvestri
Old Friends - Extended Version Howard Shore
Prologue: Book II and the Escape from the Dursleys John Williams
Fawkes the Phoenix John Williams
Harry's Wondrous World John Williams
Hogwarts' March Patrick Doyle
Twilight: River Flows In You Best Movie Soundtracks
Mombasa Hans Zimmer
Flying Theme (From "E.T. the Extra-Terrestrial") John Williams
Flying Theme (From "E.T. the Extra-Terrestrial") London Symphony Orchestra
The Social Network The Academy Orchestra
I'm Not a

Tears Of Joy Slow Club
American Daydream Electric Guest
Zombie Jamie T
If You Leave Me Now Chicago
Bear The Antlers
Pale Blue Eyes The Velvet Underground
Mad World Tears For Fears
Make It Wit Chu Queens of the Stone Age
Stuck In The Middle With You Stealers Wheel
Dirty Work Steely Dan
No Woman, No Cry - Live At The Lyceum, London/1975 Bob Marley & The Wailers
Girl, You'll Be A Woman Soon Neil Diamond
Sweet Disposition The Temper Trap
You And Me Penny & The Quarters
Lucy in the Sky with Diamonds (feat. Miley Cyrus & Moby) The Flaming Lips
Lucy in the Sky with Diamonds (feat. Miley Cyrus & Moby) Miley Cyrus
Lucy in the Sky with Diamonds (feat. Miley Cyrus & Moby) Moby
Golden Brown The Stranglers
God's Gonna Cut You Down Johnny Cash
The Sea Morcheeba
Your Rocky Spine Great Lake Swimmers
Dreamer Ozzy Osbourne
Mr. Crowley - 2002 Version Ozzy Osbourne
Stillborn Black Label Society
Shuffle Bombay Bicycle Club
Euphemia Area 11
Heaven-Piercing Giga Drill Area 11
Me Gustas Tu Manu Chao
Heartbeat

In [31]:
df2.to_csv("ten_thousand_songs.csv")

In [32]:
sp.audio_features(VideojuegosBienChingones['items'][0]['track']['uri'])

[{'danceability': 0.308,
  'energy': 0.431,
  'key': 4,
  'loudness': -11.646,
  'mode': 0,
  'speechiness': 0.0327,
  'acousticness': 0.599,
  'instrumentalness': 0.298,
  'liveness': 0.156,
  'valence': 0.181,
  'tempo': 105.807,
  'type': 'audio_features',
  'id': '2xIRBmAuVk6s7ZoukykRqS',
  'uri': 'spotify:track:2xIRBmAuVk6s7ZoukykRqS',
  'track_href': 'https://api.spotify.com/v1/tracks/2xIRBmAuVk6s7ZoukykRqS',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/2xIRBmAuVk6s7ZoukykRqS',
  'duration_ms': 166533,
  'time_signature': 3}]